In [1]:
from ib_insync import *
util.startLoop()

ib = IB()
ib.client.setConnectOptions('+PACEAPI')
ib.connect('127.0.0.1', 7497, clientId=19)

<IB connected to 127.0.0.1:7497 clientId=19>

In [2]:
import numpy as np
import yahoo_fin.stock_info as si
import requests
import pandas as pd
import datetime as dt
import os
import os.path
from dateutil.relativedelta import relativedelta

In [ ]:
inputs_path = "./../../Inputs/historical_data_inputs.csv"
data_path = "./../../HistoricalData"
interval_time_list = ['1 day', '1 hour', '10 mins']
inputs_dict = GetInputs(inputs_path)


In [ ]:
all_contracts = GetAllContractsforTickers(symbols)